# The Scottish Deanery Will Provide Digital Evidence Required for Junior Doctors to Easily Rotate Around the Scottish Healthcare System

These credentials include:

* Right To Work Credential
* DBS Check
* Compulsory Basic Training
* Immunity Certificate


![Credentials](./zoom-cred-deps.png)

## Important things to consider

* All junior doctors join a deanery. In Scotland there is only one.
* The Deanery will check both the identity documents and GMC Licence of the doctor. 
* Where necessary they will perform the proper assurance processes before providing the doctor with digital evidence in the form of a credential that these processes have been completed.

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://scottish-deanery-agent:3021 and an api key of DeaneryAPIKey


## Write DID to the Public Ledger

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.

In [3]:
public_did_response = await agent_controller.wallet.get_public_did()

In [4]:
if public_did_response["result"]:
    did_obj = public_did_response["result"]
else:
    create_did_response = await agent_controller.wallet.create_did()
    did_obj = create_did_response['result']
print("DID", did_obj)

DID {'did': 'MKDB7pQgw1KGJZ1UtJHCkY', 'verkey': 'C5AYC5nVmnPaCCWPV1qTjpZuDEoM2F53QZWkt3i3TBtq', 'posture': 'wallet_only'}


In [5]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_obj["did"],"verkey":did_obj["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "MKDB7pQgw1KGJZ1UtJHCkY": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by MKDB7pQgw1KGJZ1UtJHCkY to the ledger with role ENDORSER"}}'}


## Accept Transaction Author Agreement

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [6]:
taa_response = await agent_controller.ledger.get_taa()
TAA = taa_response['result']['taa_record']
TAA['mechanism'] = "service_agreement"
await agent_controller.ledger.accept_taa(TAA)

{}

## Assign Agent Public DID if Not Set

Will only be ran if ACAPY_WALLET_SEED not initially set.

In [7]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
print("Successfully intialised agent with Public DID : ", did_obj["did"])

Successfully intialised agent with Public DID :  MKDB7pQgw1KGJZ1UtJHCkY


## Writing Schema

Write as many schema to the ledger as you like. Be sure to store each schema_id, you will need these when it comes to authoring credential defintition transactions and issuing credentials against this schema.

Uncomment and copy the below cell as many times as you need. Be sure to update any arugments surrounded by <> with your own details.

In [8]:
# Define you schema name - must be unique on the ledger
schema_name = "UK Right to Work"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema
attributes = ["Eligible", "Date Completed", "Expiration Date", "Address"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
rtw_schema_id = response["schema_id"]

In [9]:
# Define you schema name - must be unique on the ledger
schema_name = "DBS Check"
# Can version the schema if you wish to update it
schema_version = "0.0.1"
# Define any list of attributes you wish to include in your schema

# The idea with the Certificate Reference Number is it could be looked up on the gov database to verify this information
# Rather than reproducing all the certification information which the GMC is not authorised to do
# This credential is evidence that a DBS check has taken place
attributes = ["Certificate Reference Number","Date Completed", "Expiration Date", "Type"]

response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
dbs_schema_id = response["schema_id"]

## Writing Credential Definitions

For all schema you intend to issue credentials against your agent must author a credential definition transaction to the public ledger. This specifies the public cryptographic material your agent will use to sign all credentials issued against a specific schema. 

Again uncomment and copy this cell as often as you need. Remebering to update the arguments in <> to specify your schema identifiers. Store each credential definition identifier in a unique variable.

In [10]:
# Tag and group specific credential definitions
tag = "doctor-onboarding"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = False

cred_def_response = await agent_controller.definitions.write_cred_def(rtw_schema_id, tag, support_revocation)
rtw_cred_def_id = cred_def_response["credential_definition_id"]

In [11]:
# Tag and group specific credential definitions
tag = "doctor-onboarding"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
support_revocation = False

cred_def_response = await agent_controller.definitions.write_cred_def(dbs_schema_id, tag, support_revocation)
dbs_cred_def_id = cred_def_response["credential_definition_id"]

## Persist Identifiers for use throughout other business logic notebooks associated with this agent

The schema_id and cred_def_id value pairs are required whenever issuing credentials, and also can be used to constrain acceptable proof requests. In a real application these values might be stored in environment variables or most likely in a database. For notebooks we have found it easier to store as string values in a cell and then load these values into the jupyter store so that they can be fetched across multiple notebooks.

As such you are recommended to print out each of the schema and cred def identifiers used by your agent and copy them across to your **main** business logic notebook where you should store them in a variable and save them to the jupyter store. Remember, you should only be running this notebook once so having this logic in here will not be useful.


In [12]:
print(f"rtw_schema_id = '{rtw_schema_id}'")
print(f"rtw_cred_def_id = '{rtw_cred_def_id}'")

rtw_schema_id = 'MKDB7pQgw1KGJZ1UtJHCkY:2:UK Right to Work:0.0.1'
rtw_cred_def_id = 'MKDB7pQgw1KGJZ1UtJHCkY:3:CL:216271:doctor-onboarding'


In [13]:
print(f"dbs_schema_id = '{dbs_schema_id}'")
print(f"dbs_cred_def_id = '{dbs_cred_def_id}'")

dbs_schema_id = 'MKDB7pQgw1KGJZ1UtJHCkY:2:DBS Check:0.0.1'
dbs_cred_def_id = 'MKDB7pQgw1KGJZ1UtJHCkY:3:CL:216272:doctor-onboarding'


## Terminate Controller


In [27]:
await agent_controller.terminate()